In [1]:
import tensorflow as tf

# TensorFlow Graph

In [18]:
BOARD_SIZE = 8
N_PLANES = 13
DEFAULT_N_FILTER = 256
N_ACTIONS = 8 * 8 * 73

N_POLICY_FILTER = 2
N_VALUE_FILTER = 1

N_RESIDUAL = 3

In [19]:
def convolution_block(inputs):
    conv = tf.layers.conv2d(inputs, filters=DEFAULT_N_FILTER, kernel_size=3, strides=(1, 1), padding='SAME')
    batch = tf.layers.batch_normalization(conv, training=train_ind)
    act = tf.nn.relu(batch)
    return act


def residual_block(inputs):
    conv1 = tf.layers.conv2d(inputs, filters=DEFAULT_N_FILTER, kernel_size=3, strides=(1, 1), padding='SAME')
    batch1 = tf.layers.batch_normalization(conv1, training=train_ind)
    act1 = tf.nn.relu(batch1)
    conv2 = tf.layers.conv2d(act1, filters=DEFAULT_N_FILTER, kernel_size=3, strides=(1, 1), padding='SAME')
    batch2 = tf.layers.batch_normalization(conv2, training=train_ind)
    connect = batch2 + inputs
    act2 = tf.nn.relu(connect)
    return act2


def policy_head(inputs):
    conv = tf.layers.conv2d(inputs, filters=N_POLICY_FILTER, kernel_size=1, padding='SAME')
    batch = tf.layers.batch_normalization(conv, training=train_ind)
    act = tf.nn.relu(batch)
    flat = tf.layers.flatten(act)
    policy = tf.layers.dense(flat, N_ACTIONS)
    return policy

def value_head(inputs):
    conv = tf.layers.conv2d(inputs, filters=N_VALUE_FILTER, kernel_size=1, padding='SAME')
    batch = tf.layers.batch_normalization(conv, training=train_ind)
    act1 = tf.nn.relu(batch)
    dense = tf.layers.dense(act1, 256)
    act2 = tf.nn.relu(dense)
    dense2 = tf.layers.dense(act2, 1)
    value = tf.nn.tanh(dense2)
    return value

In [21]:
tf.reset_default_graph()

state = tf.placeholder(tf.float32, [None, BOARD_SIZE, BOARD_SIZE, N_PLANES])
train_ind = tf.placeholder(tf.bool, [])

h = convolution_block(state)

for i in range(N_RESIDUAL):
    h = residual_block(h)
    
policy = policy_head(h)
value = policy_head(h)